In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 33.4MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2427622249643653293
xla_global_id: -1
]


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [0, 1, 2, 3]
LSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in LSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(LSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'LSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'LSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_1",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.08750240345185537,
  "MSE_normalize": 0.007743563884469202,
  "RMSE_normalize": 0.08799752203595963,
  "MAPE_normalize": 0.0909086890915444,
  "MAE": 7794.888392857143,
  "MSE": 61449835.68840681,
  "RMSE": 7838.994558513663,
  "MAPE": 0.05591079897081129
}
Percent CPU Usage: 10.9
Percent Ram Usage: 3.0
2022-10-12 20:42:37.448290+07:00
PROGRESS: [ 2 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_1",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 13,
  "patience_list": 3,
  "MAE_normalize": 0.15283574430105057,
  "MSE_normalize": 0.023587334726693308,
  "RMSE_normalize": 0.15358168747182493,
  "MAPE_normalize": 0.15881257826081616,
  "MAE": 13614.911830357

{
  "model": "LSTM",
  "AHB_number": "AHB_1",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 18,
  "patience_list": 3,
  "MAE_normalize": 0.24936412966776286,
  "MSE_normalize": 0.06243213410562253,
  "RMSE_normalize": 0.2498642313449897,
  "MAPE_normalize": 0.2593538630822084,
  "MAE": 22213.85379464286,
  "MSE": 495436547.2770386,
  "RMSE": 22258.40396967039,
  "MAPE": 0.15944126355948549
}
Percent CPU Usage: 15.5
Percent Ram Usage: 3.2
2022-10-12 20:42:55.422558+07:00
PROGRESS: [ 6 / 10080 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 8285.970982142857,
  "MSE": 69385391.23168945,
  "RMSE": 8329.789386994695,
  "MAPE": 0.0703896273966687
}
Percent CPU Usage: 15.7
Percent Ram Usage: 3.6
2022-10-13 14:47:08.499513+07:00
PROGRESS: [ 9873 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_4",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 53,
  "patience_list": 10,
  "MAE_normalize": 0.10295390537418288,
  "MSE_normalize": 0.010669768080720773,
  "RMSE_normalize": 0.10329456946384342,
  "MAPE_normalize": 0.1070164689549987,
  "MAE": 6640.936383928572,
  "MSE": 44394378.837916784,
  "RMSE": 6662.910688123981,
  "MAPE": 0.056433060501096456
}
Percent CPU Usage: 18.6
Percent Ram Usage: 3.6
2022-10-13 14:47:13.958151+07:00
PROGRESS: [ 9874 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_4",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_

In [ ]:
name = "all_performance_LSTM_5nsteps_AHB1-2-3-4"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_LSTM_5nsteps_AHB1-2-3-4.json /content/drive/MyDrive/Depression
! cp /content/all_performance_LSTM_5nsteps_AHB1-2-3-4.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,LSTM_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,LSTM,AHB_1,uni,1,8,0.2,2,5,12,3,0.087502,0.007744,0.087998,0.090909,7794.888393,6.144984e+07,7838.994559,0.055911
1,LSTM,AHB_1,mul,1,8,0.2,2,5,13,3,0.152836,0.023587,0.153582,0.158813,13614.911830,1.871797e+08,13681.361990,0.097667
2,LSTM,AHB_1,uni,1,8,0.2,4,5,43,3,0.083354,0.007049,0.083959,0.086566,7425.323661,5.593907e+07,7479.242562,0.053248
3,LSTM,AHB_1,mul,1,8,0.2,4,5,7,3,0.088112,0.007924,0.089017,0.091451,7849.209821,6.288224e+07,7929.832562,0.056266
4,LSTM,AHB_1,uni,1,8,0.2,8,5,18,3,0.249364,0.062432,0.249864,0.259354,22213.853795,4.954365e+08,22258.403970,0.159441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,LSTM,AHB_4,mul,3,128,0.5,32,5,40,10,0.024000,0.000613,0.024751,0.024838,1548.071429,2.548989e+06,1596.555351,0.013125
10076,LSTM,AHB_4,uni,3,128,0.5,64,5,44,10,0.093632,0.008780,0.093704,0.097460,6039.640625,3.653345e+07,6044.291031,0.051360
10077,LSTM,AHB_4,mul,3,128,0.5,64,5,39,10,0.019805,0.000406,0.020155,0.020721,1277.504464,1.690241e+06,1300.092501,0.010893
10078,LSTM,AHB_4,uni,3,128,0.5,128,5,78,10,0.014953,0.000232,0.015218,0.015633,964.542411,9.635825e+05,981.622398,0.008221
